In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os

# Acquire

In [2]:
def get_311_data():
    '''
    This function uses pandas read .csv to read in the downloaded .csv 
    from: https://data.sanantonio.gov/dataset/service-calls/resource/20eb6d22-7eac-425a-85c1-fdb365fd3cd7
    after the .csv is read in, it returns it as a data frame.
    '''
    df= pd.read_csv('service_calls.csv')
    return df


In [3]:
df= get_311_data()

In [4]:
df.head()

,Category,CASEID,OPENEDDATETIME,SLA_Date,CLOSEDDATETIME,Late (Yes/No),Dept,REASONNAME,TYPENAME,CaseStatus,SourceID,OBJECTDESC,Council District,XCOORD,YCOORD,Report Starting Date,Report Ending Date
0,Graffiti,1010444245,8/15/2012,8/30/2012,NaN,YES,Code Enforcement Services,Graffiti,Graffiti Public Property,Open,Web Portal,"600 NOGALITOS ST, San Antonio, 78204",5,2125683.0,13695548.0,5/15/2020,5/15/2021
1,Property Maintenance,1010888252,6/6/2013,8/9/2013,NaN,YES,Code Enforcement Services,Code Enforcement (IntExp),Alley-Way Maintenance,Open,Web Portal,"6043 CASTLE QUEEN, San Antonio, 78218",2,2169702.0,13725769.0,5/15/2020,5/15/2021
2,Property Maintenance,1010966128,7/19/2013,9/23/2013,NaN,YES,Code Enforcement Services,Code Enforcement (IntExp),Junk Vehicle On Private Property,Open,Web Portal,"842 KIRK PL, San Antonio, 78226",5,2116192.0,13692260.0,5/15/2020,5/15/2021
3,Property Maintenance,1011052825,9/16/2013,9/30/2013,NaN,YES,Code Enforcement Services,Code Enforcement,Right Of Way/Sidewalk Obstruction,Open,Internal Services Requests,"54 KENROCK RIDGE, San Antonio, 78254",7,2082242.0,13737817.0,5/15/2020,5/15/2021
4,Property Maintenance,1011052826,9/16/2013,9/30/2013,NaN,YES,Code Enforcement Services,Code Enforcement,Right Of Way/Sidewalk Obstruction,Open,Internal Services Requests,"74 KENROCK RIDGE, San Antonio, 78254",7,2082389.0,13737877.0,5/15/2020,5/15/2021


In [5]:
df.Dept.value_counts()

Solid Waste Management       190658
Development Services         132826
Animal Care Services          75222
Trans & Cap Improvements      33684
Code Enforcement Services      5294
Customer Service               4196
Metro Health                   2827
Parks and Recreation            433
City Council                      1
Name: Dept, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495440 entries, 0 to 495439
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Category              495440 non-null  object 
 1   CASEID                495440 non-null  int64  
 2   OPENEDDATETIME        495440 non-null  object 
 3   SLA_Date              481300 non-null  object 
 4   CLOSEDDATETIME        441387 non-null  object 
 5   Late (Yes/No)         495440 non-null  object 
 6   Dept                  445141 non-null  object 
 7   REASONNAME            495440 non-null  object 
 8   TYPENAME              495440 non-null  object 
 9   CaseStatus            495440 non-null  object 
 10  SourceID              495440 non-null  object 
 11  OBJECTDESC            495440 non-null  object 
 12  Council District      495440 non-null  int64  
 13  XCOORD                495345 non-null  float64
 14  YCOORD                495345 non-null  float64
 15  

In [7]:
df.describe()

,CASEID,Council District,XCOORD,YCOORD
count,4.954400e+05,495440.000000,4.953450e+05,4.953450e+05
mean,1.016457e+09,4.761099,2.120971e+06,1.371271e+07
std,3.836717e+05,2.809984,2.666102e+04,2.715970e+04
min,1.010444e+09,0.000000,2.029622e+06,1.359464e+07
25%,1.016290e+09,2.000000,2.104087e+06,1.369342e+07
50%,1.016462e+09,5.000000,2.121784e+06,1.370894e+07
75%,1.016691e+09,7.000000,2.139919e+06,1.373111e+07
max,1.016871e+09,10.000000,2.235998e+06,1.381636e+07


In [8]:
df.isna().sum()

Category                    0
CASEID                      0
OPENEDDATETIME              0
SLA_Date                14140
CLOSEDDATETIME          54053
Late (Yes/No)               0
Dept                    50299
REASONNAME                  0
TYPENAME                    0
CaseStatus                  0
SourceID                    0
OBJECTDESC                  0
Council District            0
XCOORD                     95
YCOORD                     95
Report Starting Date        0
Report Ending Date          0
dtype: int64

#### Takeaways:
    - Most columns will need names cleaned up for clarity/easier to call later
    - The nans in the X/Y Coord column can be dropped because there are only 95 instances
    - The nans in the closeddatetime are likely work orders that still have not been fixed.
        - This is significant because there are 54,053 instances of having an open work order
    - There are 50,299 instances of an empty department, perhaps using typename or category
    to "educated guess" what department the call is covered by
    

# Prepare

- Rename
    - 'Late (Yes/No) to just 'late'
    - SLA_date to 'due_date'
    - YCOORD to 'latitude'
    - XCOORD to 'longitude'

In [9]:
def clean_column_names(df):
    '''This function reads in a dataframe as a positional argument, makes the column names easier to call and
    more python friendly. It also extracts the zip code from the address column. It then returns a cleaned data 
    frame.'''
    df= df.rename(columns={
                    'Category':'category', 'OPENEDDATETIME':'open_date', 'Dept': 'dept',
                    'SLA_Date':'due_date', 'CLOSEDDATETIME': 'closed_date', 'Late (Yes/No)': 'is_late',
                    'OBJECTDESC': 'address', 'REASONNAME': 'call_reason', 'TYPENAME': 'case_type', 
                    'Council District': 'council_district', 'CASEID': 'case_id',
                    'CaseStatus': 'case_status', 'SourceID':'source_id', 'XCOORD': 'longitude', 'YCOORD': 'latitude',
                    'Report Starting Date': 'report_start_date', 'Report Ending Date': 'report_end_date'
                      })
    df['zipcode'] = df['address'].str.extract(r'(\d{5}\-?\d{0,4})')
    return df
    

In [10]:
df= clean_column_names(df)

In [11]:
df.head()

,category,case_id,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,council_district,longitude,latitude,report_start_date,report_end_date,zipcode
0,Graffiti,1010444245,8/15/2012,8/30/2012,NaN,YES,Code Enforcement Services,Graffiti,Graffiti Public Property,Open,Web Portal,"600 NOGALITOS ST, San Antonio, 78204",5,2125683.0,13695548.0,5/15/2020,5/15/2021,78204
1,Property Maintenance,1010888252,6/6/2013,8/9/2013,NaN,YES,Code Enforcement Services,Code Enforcement (IntExp),Alley-Way Maintenance,Open,Web Portal,"6043 CASTLE QUEEN, San Antonio, 78218",2,2169702.0,13725769.0,5/15/2020,5/15/2021,78218
2,Property Maintenance,1010966128,7/19/2013,9/23/2013,NaN,YES,Code Enforcement Services,Code Enforcement (IntExp),Junk Vehicle On Private Property,Open,Web Portal,"842 KIRK PL, San Antonio, 78226",5,2116192.0,13692260.0,5/15/2020,5/15/2021,78226
3,Property Maintenance,1011052825,9/16/2013,9/30/2013,NaN,YES,Code Enforcement Services,Code Enforcement,Right Of Way/Sidewalk Obstruction,Open,Internal Services Requests,"54 KENROCK RIDGE, San Antonio, 78254",7,2082242.0,13737817.0,5/15/2020,5/15/2021,78254
4,Property Maintenance,1011052826,9/16/2013,9/30/2013,NaN,YES,Code Enforcement Services,Code Enforcement,Right Of Way/Sidewalk Obstruction,Open,Internal Services Requests,"74 KENROCK RIDGE, San Antonio, 78254",7,2082389.0,13737877.0,5/15/2020,5/15/2021,78254
